# How to Implement the Backpropagation Algorithm From Scratch in Python

Based on the tutorial by Jason Brownlee — [Machine Learning Mastery](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/)

---

**Backpropagation** is a supervised learning algorithm for training multilayer feed-forward neural networks. It works by propagating the error signal backward through the network and using **gradient descent** to update weights.

In this notebook we will:
- Initialize a neural network
- Forward propagate inputs
- Backpropagate errors
- Train the network with stochastic gradient descent
- Make predictions
- Apply the algorithm to the Wheat Seeds dataset

## Imports

In [ ]:
from math import exp
from random import seed, random
from csv import reader
from random import randrange

---
## 1. Initialize Network

We represent a network as a **list of layers**, where each layer is a **list of neurons**. Each neuron is a dictionary containing its `weights` (one per input + one bias) and other training properties.

The network architecture:
- **Input layer**: the raw feature values (not stored explicitly)
- **Hidden layer**: `n_hidden` neurons, each connected to all inputs
- **Output layer**: one neuron per class

Weights are initialized **randomly** in the range `[0, 1)`.

In [ ]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    """
    Create and initialize a new neural network.
    Each neuron: {'weights': [w1, w2, ..., bias]}
    """
    network = []
    # Hidden layer: n_hidden neurons, each with (n_inputs + 1) weights (last = bias)
    hidden_layer = [{'weights': [random() for _ in range(n_inputs + 1)]} for _ in range(n_hidden)]
    network.append(hidden_layer)
    # Output layer: n_outputs neurons, each with (n_hidden + 1) weights
    output_layer = [{'weights': [random() for _ in range(n_hidden + 1)]} for _ in range(n_outputs)]
    network.append(output_layer)
    return network

In [ ]:
# Example: 2 inputs, 1 hidden neuron, 2 outputs
seed(1)
network = initialize_network(2, 1, 2)
for layer_idx, layer in enumerate(network):
    print(f"Layer {layer_idx + 1}:")
    for neuron in layer:
        print(f"  Neuron weights: {neuron['weights']}")

---
## 2. Forward Propagation

Forward propagation computes the **output of each neuron** layer by layer until we reach the final output.

### 2.1 Neuron Activation

The activation of a neuron is the **weighted sum** of inputs plus a bias:

$$\text{activation} = \sum_{i=1}^{n} w_i \cdot x_i + b$$

where $w_i$ are the weights, $x_i$ are the inputs, and $b$ is the bias term.

In [ ]:
def activate(weights, inputs):
    """
    Calculate neuron activation: dot(weights, inputs) + bias
    The last weight is the bias (paired with a constant input of 1).
    """
    activation = weights[-1]  # bias
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

### 2.2 Sigmoid Transfer Function

After computing the activation, we pass it through an **activation function** to introduce non-linearity. We use the **sigmoid function**:

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

The sigmoid maps any real number to the range $(0, 1)$, making it ideal for classification problems.

In [ ]:
def transfer(activation):
    """
    Sigmoid activation function: σ(x) = 1 / (1 + e^(-x))
    Output is always in range (0, 1).
    """
    return 1.0 / (1.0 + exp(-activation))

### 2.3 Forward Propagation Through Layers

The output of each layer becomes the **input to the next layer**. For each neuron, we compute the activation and store the output as `'output'`.

In [ ]:
def forward_propagate(network, row):
    """
    Forward propagate one row through the network.
    Returns the final output layer activations.
    """
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs  # outputs of this layer become inputs to next
    return inputs

In [ ]:
# Test forward propagation
seed(1)
network = initialize_network(2, 1, 2)
row = [1, 0, None]  # 2 inputs + class label placeholder
output = forward_propagate(network, row)
print("Network output:", output)

---
## 3. Backpropagation

Backpropagation calculates the **error gradient** for each neuron and propagates it backward through the network to update the weights.

### 3.1 Sigmoid Derivative

To update weights using gradient descent, we need the **derivative of the sigmoid function**. Given that we already have the sigmoid output $\sigma(x)$:

$$\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))$$

This is efficient since we already stored the output from forward propagation.

In [ ]:
def transfer_derivative(output):
    """
    Derivative of the sigmoid function.
    σ'(x) = σ(x) * (1 - σ(x))
    We use the already-computed output to avoid recalculation.
    """
    return output * (1.0 - output)

### 3.2 Computing Error Signals (Delta)

**Output layer error** — The delta for each output neuron is:

$$\delta^{\text{out}}_j = (\hat{y}_j - y_j) \cdot \sigma'(\text{out}_j)$$

where $\hat{y}_j$ is the predicted output and $y_j$ is the expected output (one-hot encoded).

**Hidden layer error** — The delta is the weighted sum of errors from the next layer:

$$\delta^{\text{hidden}}_j = \left(\sum_k w_{jk} \cdot \delta^{\text{out}}_k\right) \cdot \sigma'(\text{out}_j)$$

In [ ]:
def backward_propagate_error(network, expected):
    """
    Backpropagate error and store delta in each neuron.
    'expected' is the one-hot encoded expected output.
    """
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []

        if i != len(network) - 1:
            # Hidden layers: weighted sum of errors from next layer
            for j in range(len(layer)):
                error = sum(neuron['weights'][j] * neuron['delta'] for neuron in network[i + 1])
                errors.append(error)
        else:
            # Output layer: difference between prediction and expected
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])

        # Compute delta = error * sigmoid_derivative(output)
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

### 3.3 Update Weights

Once we have the delta for each neuron, we update the weights using **stochastic gradient descent**:

$$w_i \leftarrow w_i - \eta \cdot \delta \cdot x_i$$

where $\eta$ is the **learning rate** and $x_i$ is the input to the neuron (output of the previous layer). The bias weight is updated with $x_i = 1$.

In [ ]:
def update_weights(network, row, l_rate):
    """
    Update weights using gradient descent.
    w_i = w_i - learning_rate * delta * input_i
    """
    for i in range(len(network)):
        # Input to this layer: original row (for hidden) or previous layer outputs
        inputs = row[:-1]  # exclude class label
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']  # update bias

---
## 4. Train Network

Training is done using **Stochastic Gradient Descent (SGD)**: for each epoch, we iterate over all training examples one at a time, forward propagate, backpropagate the error, and update weights.

The **sum of squared errors (SSE)** is accumulated each epoch to monitor training progress:

$$\text{SSE} = \sum_{j} (\hat{y}_j - y_j)^2$$

In [ ]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    """
    Train a network using stochastic gradient descent.

    Parameters:
        network   : initialized network
        train     : training dataset (last column = class label as integer)
        l_rate    : learning rate η
        n_epoch   : number of training epochs
        n_outputs : number of output neurons (= number of classes)
    """
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            # One-hot encode the expected output
            expected = [0] * n_outputs
            expected[int(row[-1])] = 1
            # Accumulate SSE
            sum_error += sum((expected[j] - outputs[j])**2 for j in range(len(expected)))
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f"Epoch={epoch+1:4d}  learning_rate={l_rate:.3f}  error={sum_error:.3f}")

In [ ]:
# Test training on a small XOR-like dataset
seed(1)
dataset = [
    [2.7810836,  2.550537003, 0],
    [1.465489372, 2.362125076, 0],
    [3.396561688, 4.400293529, 0],
    [1.38807019,  1.850220317, 0],
    [3.06407232,  3.005305973, 0],
    [7.627531214, 2.759262235, 1],
    [5.332441248, 2.088626775, 1],
    [6.922596716, 1.77106367,  1],
    [8.675418651, -0.242068655, 1],
    [7.673756466, 3.508563011, 1]
]

n_inputs  = len(dataset[0]) - 1  # 2
n_outputs = len(set(row[-1] for row in dataset))  # 2

network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, l_rate=0.5, n_epoch=20, n_outputs=n_outputs)

print("\nFinal weights:")
for layer in network:
    print(layer)

---
## 5. Predict

After training, we can make predictions. We forward propagate a row and return the **index of the output neuron with the highest activation** (i.e., `argmax`):

$$\hat{y} = \arg\max_j \; \text{output}_j$$

In [ ]:
def predict(network, row):
    """
    Make a prediction for a given row.
    Returns the index of the output neuron with the highest output (argmax).
    """
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [ ]:
# Test predictions on training data
for row in dataset:
    prediction = predict(network, row)
    actual = int(row[-1])
    status = "✓" if prediction == actual else "✗"
    print(f"Expected={actual}  Predicted={prediction}  {status}")

---
## 6. Seeds Dataset Case Study

We now apply the full algorithm to a **real-world dataset** — the [Wheat Seeds Dataset](https://archive.ics.uci.edu/ml/datasets/seeds).

- **7 input features**: geometric properties of wheat kernels (area, perimeter, compactness, etc.)
- **3 output classes**: varieties of wheat (Kama=0, Rosa=1, Canadian=2)
- **210 samples** total

We evaluate using **5-fold cross-validation** with the following hyperparameters:
- Learning rate: `0.3`
- Epochs: `500`
- Hidden neurons: `5`

### 6.1 Helper Utilities

In [ ]:
def load_csv(filename):
    """Load a CSV file and return all rows as a list of lists."""
    dataset = []
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


def str_column_to_float(dataset, column):
    """Convert a column of strings to floats."""
    for row in dataset:
        row[column] = float(row[column].strip())


def str_column_to_int(dataset, column):
    """Convert a string column to integer class labels (0-indexed)."""
    class_values = sorted(set(row[column] for row in dataset))
    lookup = {val: i for i, val in enumerate(class_values)}
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

### 6.2 Normalization

We **normalize** each feature to the range $[0, 1]$ using **min-max scaling**:

$$x'_i = \frac{x_i - x_{\min}}{x_{\max} - x_{\min}}$$

This prevents features with large values from dominating the gradient updates.

In [ ]:
def dataset_minmax(dataset):
    """Find min and max values for each column."""
    minmax = []
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        minmax.append([min(col_values), max(col_values)])
    return minmax


def normalize_dataset(dataset, minmax):
    """Normalize each feature to range [0, 1] using min-max scaling."""
    for row in dataset:
        for i in range(len(row) - 1):  # exclude label column
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

### 6.3 Cross-Validation

In [ ]:
def cross_validation_split(dataset, n_folds):
    """Split dataset into k folds for cross-validation."""
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


def accuracy_metric(actual, predicted):
    """Calculate classification accuracy as a percentage."""
    correct = sum(1 for a, p in zip(actual, predicted) if a == p)
    return correct / len(actual) * 100.0


def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    """Evaluate algorithm using k-fold cross-validation."""
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = [row for f in folds for row in f if f is not fold]
        test_set  = [list(row) for row in fold]  # copy so labels can be removed
        predicted = algorithm(train_set, test_set, *args)
        actual    = [row[-1] for row in fold]
        scores.append(accuracy_metric(actual, predicted))
    return scores

### 6.4 Full Backpropagation Algorithm

In [ ]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    """
    Full backpropagation algorithm:
      1. Initialize network
      2. Train with SGD
      3. Predict on test set
    """
    n_inputs  = len(train[0]) - 1
    n_outputs = len(set(row[-1] for row in train))
    network   = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = [predict(network, row) for row in test]
    return predictions

### 6.5 Download and Prepare the Seeds Dataset

In [ ]:
import urllib.request

# Download the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv'
urllib.request.urlretrieve(url, 'wheat-seeds.csv')
print("Dataset downloaded successfully.")

### 6.6 Run Cross-Validation and Report Results

In [ ]:
seed(1)

# Load and preprocess dataset
dataset = load_csv('wheat-seeds.csv')
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
str_column_to_int(dataset, len(dataset[0]) - 1)  # convert class labels

# Normalize features
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Hyperparameters
n_folds  = 5
l_rate   = 0.3
n_epoch  = 500
n_hidden = 5

# Evaluate
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

print("\n" + "="*40)
print(f"Fold Scores: {[f'{s:.1f}%' for s in scores]}")
print(f"Mean Accuracy: {sum(scores)/len(scores):.2f}%")

## 7. Survival Prediction Titanic

---
## Summary

| Step | What it does |
|------|--------------|
| **1. Initialize Network** | Random weights for each neuron (including bias) |
| **2. Forward Propagation** | Compute neuron outputs layer by layer using sigmoid |
| **3. Backpropagation** | Compute error deltas from output → input using chain rule |
| **4. Train Network** | SGD: repeat forward + backward pass for multiple epochs |
| **5. Predict** | argmax of output layer activations |
| **6. Case Study** | ~93–96% accuracy on Wheat Seeds dataset with 5-fold CV |

### Key Formulas

| Formula | Expression |
|---------|------------|
| Activation | $\displaystyle a = \sum_i w_i x_i + b$ |
| Sigmoid | $\displaystyle \sigma(x) = \frac{1}{1+e^{-x}}$ |
| Sigmoid derivative | $\sigma'(x) = \sigma(x)(1-\sigma(x))$ |
| Output delta | $\delta^{\text{out}} = (\hat{y} - y) \cdot \sigma'(\text{out})$ |
| Hidden delta | $\delta^{\text{hid}} = \left(\sum_k w_k \delta_k^{\text{out}}\right) \cdot \sigma'(\text{out})$ |
| Weight update | $w_i \leftarrow w_i - \eta \cdot \delta \cdot x_i$ |